<a href="https://colab.research.google.com/github/manan180796/GNN/blob/master/Node_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
# !pip install -q torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
# !pip install -q torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
# !pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

In [ ]:
!rm -rf runs/*

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(comment="hidden_channels=[200,32],dropout=0.5")

In [ ]:
from torch_geometric.nn import GCNConv
# from torch.nn import Linear

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features, 200)
        self.conv2 = GCNConv(200, 32)
        self.conv3 = GCNConv(32, dataset.num_classes)
        # self.final = Linear(dataset.num_classes,dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        return x

In [ ]:
gcn_model = GCN(hidden_channels=[200,32])
writer.add_graph(model=gcn_model,input_to_model=(data.x,data.edge_index))
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

def train_gcn():
      gcn_model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = gcn_model(data.x,data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss, val_loss

def test_gcn(mask):
      gcn_model.eval()
      out = gcn_model(data.x,data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc

for epoch in range(800):
    loss, val_loss = train_gcn()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    writer.add_scalars(main_tag="GCN/loss",tag_scalar_dict={"train":loss/data.train_mask.sum(),"validation":val_loss/data.val_mask.sum()},global_step=epoch)
    writer.add_scalars(main_tag="GCN/accuracy",tag_scalar_dict={"train":test_gcn(data.train_mask),"validation":test_gcn(data.val_mask)},global_step=epoch)
# writer.add_hparams(hparam_dict=)

In [ ]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
gcn_model.eval()
out = gcn_model(data.x,data.edge_index)
writer.add_embedding(mat=out,metadata=data.y,tag="GCN")

In [ ]:
writer.close()

In [ ]:
# %load_ext tensorboard
%tensorboard --logdir runs 